In [1]:
import pandas as pd
import numpy as np
import numpy_financial as npf 


In [ ]:
# Importing file
df = pd.read_excel("./data_lake/SimpleAmortizationTest.xlsx", "LoanDataTape")

# Convert data to Dict type object
headers = [header for header in df.keys()]
json_obj = []

for obj in range(1,len(df)):
    data = {}
    for head in headers:
        data[head.strip()] = df[head][obj]
    json_obj.append(data)


   loan number loan amount          interest_rate           start_date  \
0          int       float   float: (annual rate)                 date   
1            1       35000                   0.08  2023-09-01 00:00:00   
2            2       40000                   0.08  2023-10-01 00:00:00   
3            3       40000                   0.08  2023-11-01 00:00:00   
4            4       40000                   0.08  2023-12-01 00:00:00   
5            5       40000                   0.08  2024-01-01 00:00:00   
6            6       40000                   0.08  2024-02-01 00:00:00   
7            7       40000                   0.08  2024-03-01 00:00:00   
8            8       40000                   0.08  2024-04-01 00:00:00   
9            9       40000                   0.08  2024-05-01 00:00:00   
10          10       40000                   0.08  2024-06-01 00:00:00   

             term  payment frequency CPR (Conditional Prepayment Rate)  
0   float (months)               str  

In [3]:
# Calculate monthly_interest_rate, monthly_payment, SSM  and put in new dataframe

for instance in json_obj:
    rate = instance["interest_rate"]/12
    cpr = instance["CPR (Conditional Prepayment Rate)"]
    instance["monthly_interest_rate"] = rate
    monthly_payment = npf.pmt(rate, instance["term"], instance["loan amount"])
    instance["monthly_payment"] = round(monthly_payment.item(), 2) * -1
    ssm = 1-(np.float_power(1-cpr, (1/12)))
    instance["SSM"] = ssm
    print(instance)



{'loan number': 1, 'loan amount': 35000, 'interest_rate': 0.08, 'start_date': datetime.datetime(2023, 9, 1, 0, 0), 'term': 36, 'payment frequency': 'Monthly', 'CPR (Conditional Prepayment Rate)': 0.05, 'monthly_interest_rate': 0.006666666666666667, 'monthly_payment': 1096.77, 'SSM': np.float64(0.004265318777560645)}
{'loan number': 2, 'loan amount': 40000, 'interest_rate': 0.08, 'start_date': datetime.datetime(2023, 10, 1, 0, 0), 'term': 12, 'payment frequency': 'Monthly', 'CPR (Conditional Prepayment Rate)': 0.05, 'monthly_interest_rate': 0.006666666666666667, 'monthly_payment': 3479.54, 'SSM': np.float64(0.004265318777560645)}
{'loan number': 3, 'loan amount': 40000, 'interest_rate': 0.08, 'start_date': datetime.datetime(2023, 11, 1, 0, 0), 'term': 48, 'payment frequency': 'Monthly', 'CPR (Conditional Prepayment Rate)': 0.05, 'monthly_interest_rate': 0.006666666666666667, 'monthly_payment': 976.52, 'SSM': np.float64(0.004265318777560645)}
{'loan number': 4, 'loan amount': 40000, 'int

In [4]:
# Create amortization schedule

from dateutil.relativedelta import relativedelta

all_schedule = []
for obj in json_obj:
    schedule_list = []
    closing_balance = obj["loan amount"]
    amortization_date = obj["start_date"]
    for x in range(0, obj["term"]+1):
        period = x
        schedule_date = amortization_date if x==0 else amortization_date + relativedelta(months=x)
        opening_balance = 0 if x==0 else closing_balance # C
        interest_rate = obj["interest_rate"] if opening_balance > 0 else 0 # F
        month_interest_rate = obj["monthly_interest_rate"] if opening_balance > 0 else 0 # G
        interest = opening_balance * month_interest_rate if opening_balance else 0 # H
        payment = min(obj["monthly_payment"], round(opening_balance + interest, 2)) # D
        
        prepayment = round(obj ["SSM"] * opening_balance, 2) if payment <= opening_balance else 0 # E
        principal = payment + prepayment - interest
        if x > 0: 
            closing_balance = opening_balance - principal if (opening_balance - principal) > 0 else 0 
        else :
            closing_balance

        schedule = {
            "period":period,
            "date":schedule_date,
            "opening_balance":round(opening_balance,2),
            "payment":payment,
            "prepayment":prepayment,
            "interest_rate":"{:.2%}".format(round(interest_rate, 2)),
            "monthly_interest_rate":"{:.2%}".format(month_interest_rate),
            "interest":round(interest,2),
            "principal":round(principal, 2),
            "closing_balance":round(closing_balance, 2),
            "loan_number" : obj["loan number"]
        }
        all_schedule.append(schedule)
    # all_schedule[obj["loan number"]] = schedule_list

# [print(line) for line in all_schedule[1]]

agg_df = pd.DataFrame.from_dict(all_schedule)
agg_df["date"] = pd.to_datetime(agg_df["date"])
sorted_df = agg_df.sort_values(by="date", ascending=True)
sorted_df.to_csv("Simple amortization loan.csv", index=False)
# print(agg_df.date)


